In [ ]:
%load_ext autoreload
%autoreload 2

%config IPCompleter.greedy=True

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import h5py
from astropy import units
from pathlib import Path
import os
import time

import snapshot_obj
import dataset_compute
import curve_fit

import importlib

In [ ]:
importlib.reload(snapshot_obj)
importlib.reload(dataset_compute)
importlib.reload(curve_fit)

# Massive compact non-luminous halos

Here, we inspect the properties of the clump of subhalos in the $\Lambda$CDM simulation with $v_\text{max} \sim v_\text{1kpc} > 35 \text{km/s}$.

In [ ]:
snap = snapshot_obj.Snapshot("V1_LR_fix", 127)
max_point = snap.get_subhalos("Max_Vcirc", "Extended")
vmax = max_point[:,0] * units.cm.to(units.km)
v1kpc = snap.get_subhalos("V1kpc", "Extended") * units.cm.to(units.km)

In [ ]:
masks_sat, mask_isol = dataset_compute.split_satellites_by_group_number(
    snap, (1,0), (2,0))
mask_dark = dataset_compute.split_luminous(snap)[1]
mask_pecul = np.logical_and(vmax > 35, np.abs(vmax-v1kpc)/vmax < 0.1)
mask_pecul = np.logical_and.reduce([mask_isol, mask_dark, mask_pecul])
mask_ref = np.logical_and(vmax > 30, vmax < 50)
mask_ref = np.logical_and.reduce([mask_isol, mask_dark, mask_ref])

In [ ]:
masstype = snap.get_subhalos("MassType") * units.g.to(units.Msun)
for m_arr in masstype[mask_pecul]:
    print(["{:.2E}".format(m) for m in m_arr])

## Plot the rotation curves

Let us first see how many there are of these peculiar halos, and compare that to the number of all halos in the mass range:

In [ ]:
print(np.sum(mask_pecul))
print(np.sum(mask_ref))
print(np.sum(np.logical_and(mask_pecul, mask_ref)))

We see that there seem to be two kinds of rotation curves in the figure belos -- very compact ones, which constitute the clump in the V1kpc plots, and more regular looking ones.

In [ ]:
rot_curves = snap.get_subhalos('Vcirc', group='Extended/RotationCurve/All')
sub_offset = snap.get_subhalos('SubOffset', group='Extended/RotationCurve/All')

fig, axes = plt.subplots()
axes.set_xlim(-5, 20)

idxs = np.arange(mask_pecul.size)[mask_ref]
for idx in idxs:
    v_circ = rot_curves[sub_offset[idx]:sub_offset[idx+1],0] * units.cm.to(units.km)
    radii = rot_curves[sub_offset[idx]:sub_offset[idx+1],1] * units.cm.to(units.kpc)
    axes.plot(radii, v_circ, c='black')

idxs = np.arange(mask_pecul.size)[mask_pecul]
for idx in idxs:
    v_circ = rot_curves[sub_offset[idx]:sub_offset[idx+1],0] * units.cm.to(units.km)
    radii = rot_curves[sub_offset[idx]:sub_offset[idx+1],1] * units.cm.to(units.kpc)
    axes.plot(radii, v_circ, c='gray') 
    
#plt.savefig("odd_dark_compact_rotation_curves.png")

In [ ]:
LG = dataset_compute.compute_LG_centre(snap, (1,0), (2,0))
dist_to_lg = dataset_compute.distance_to_point(snap, LG) * units.cm.to(units.kpc)
print(dist_to_lg[mask_pecul])

In [ ]:
cum_mass, radii = dataset_compute.mass_accumulation_to_array(snap)
radii = radii[mask_pecul] * units.cm.to(units.kpc)

In [ ]:
print(type(radii))
print(type(radii[0]))
print(type(radii[0][0]))
print(radii[0][0][0])


In [ ]:
mass = snap.get_subhalos("Mass")[mask_pecul] * units.g.to(units.Msun)
for r, m in zip(radii, mass):
    print("{:.2E}".format(m))
    print(r.shape)
    print(type(r[1]))
    mask = (r < 1)
    print(mask)
    print(r[r < 1])

In [ ]:
cm, r = dataset_compute.compute_mass_accumulation(snap, part_type=[1])
print(type(cm), type(r))
print(len(cm), len(r))
print(type(cm[0]), type(r[0]))
print(cm[0].size, r[0].size)
print(type(cm[0][0]), type(r[0][0]))

In [ ]:
cm_data = snap.get_subhalos("MassAccum", group="Extended")
print(type(cm_data))
print(cm_data.shape)

In [ ]:
pts = snap.get_particles("PartType", part_type=[0,1,2,3,4,5])
masses = snap.get_particles("Masses", part_type=[0,1,2,3,4,5])

for pt in [0,1,2,3,4,5]:
    mask_pt = (pts == pt)
    print(np.sum(mask_pt))

    print(np.all(masses[mask_pt] == masses[mask_pt][0]))

In [ ]:
pts = snap.get_particles("PartType", part_type=[0,1,4,5])
sum = 0
for pt in [0,1,4,5]:
    mask_pt = (pts == pt)
    grouped_masses = dataset_compute.group_selected_particles_by_subhalo(snap, "Masses", selection_mask=mask_pt)
    n = np.concatenate(grouped_masses["Masses"]).size
    sum += n
    print(n)
print(sum)

In [ ]:
part_type = [0,1,4,5]
grouped_data = dataset_compute.group_selected_particles_by_subhalo(
    snap, 'Coordinates', 'Masses', part_type=part_type)
print(np.concatenate(grouped_data["Masses"]).size)

In [ ]:
pts = {'all': None, 'gas': [0], 'dm': [1]}
v_circ = {}
radii = {}

for key, pt in pts.items():
    if pt is None:
        v, r = dataset_compute.compute_rotation_curves(snap, n_soft=1)
    else:
        v, r = dataset_compute.compute_rotation_curves(snap, n_soft=1, part_type=pt)

    v_circ[key] = v[mask_pecul] * units.cm.to(units.km)
    radii[key] = r[mask_pecul] * units.cm.to(units.kpc)
    
for pt in pts.keys():
    print(pt)
    for r in radii[pt]:
        print(np.sum(r < 1), np.sum(r < 2))
    print('')

In [ ]:
fig, axes = plt.subplots()

axes.set_xlim(-2, 20)

for v, r in zip(v_circ['dm'], radii['dm']):
    axes.plot(r, v, c='grey') 
    
for v, r in zip(v_circ['gas'], radii['gas']):
    axes.plot(r, v, c='black') 